# Proyecto 2: Obtención de eigenvalores de un sistema de ecuaciones

# Preeliminares: Elementos Finitos

## Bibliotecas requeridas

Se requiere de
* **Numpy**:
 Para el manejo de vectores y matrices, junto a procedimientos que involucren este tipo de información

* **Mathplotlib**:
  Se va a utilizar para visualizar la comparación entre los datos de la solución analítica.

* **Math**:
  Para cualquier operación que se vaya a utilizar

* **Pandas**:
  Para generar tablas, en este caso, para mostrar resultados

* **Scipy.special**:
  Permite calcular de forma eficiente y precisa los nodos y pesos de Gauss-Legendre [Nos sirve para comparar]

In [74]:
import numpy as np
import matplotlib.pyplot as plt
import math as mt
import pandas as pd
from scipy.special import p_roots
import scipy.linalg as la



## Raíces

Calcula los nodos (raíces) y los pesos de la cuadratura de Gauss-Legendre para un polinomio de grado `q`.

La cuadratura de Gauss-Legendre se utiliza para evaluar integrales aproximadas con alta precisión al emplear puntos y pesos óptimos definidos sobre el intervalo $[-1, 1]$

**Entrada:**
* `q :int`

Grado del polinomio para el cual se desean calcular las raíces y los pesos. Debe estar en el rango $1 \leq q \leq$ 7.

**Salida:**

* `xq : numpy.ndarray`

Un arreglo de `q` valores que representan las raíces (nodos) del polinomio de Legendre en el intervalo $[-1, 1]$.
    
* `wq : numpy.ndarray`

Un arreglo de `q` valores que representan los pesos asociados a cada raíz, utilizados para calcular la integral.


In [61]:
def p_roots(q):
  if q==1:
    xq = np.array([0])
    wq=np.array([2])
  elif q == 2:
        xq = np.array([-0.5773502692, 0.5773502692])
        wq = np.array([1.0, 1.0])
  elif q == 3:
        xq = np.array([-0.7745966692, 0.0, 0.7745966692])
        wq = np.array([0.5555555556, 0.8888888889, 0.5555555556])
  elif q == 4:
        xq = np.array([-0.8611363116, -0.339981044, 0.339981044, 0.8611363116])
        wq = np.array([0.3478548451, 0.6521451549, 0.6521451549, 0.3478548451])
  elif q == 5:
        xq = np.array([-0.9061798459, -0.5384693101, 0.0, 0.5384693101, 0.9061798459])
        wq = np.array([0.2369268851, 0.4786286705, 0.5688888889, 0.4786286705, 0.2369268851])
  elif q == 6:
        xq = np.array([-0.9324695142, -0.6612093865, -0.2386191861, 0.2386191861, 0.6612093865, 0.9324695142])
        wq = np.array([0.1713244924, 0.3607615730, 0.4679139346, 0.4679139346, 0.3607615730, 0.1713244924])
  elif q == 7:
        xq = np.array([-0.9491079123, -0.7415311856, -0.4058451514, 0.0, 0.4058451514, 0.7415311856, 0.9491079123])
        wq = np.array([0.1294849662, 0.2797053915, 0.3818300505, 0.4179591837, 0.3818300505, 0.2797053915, 0.1294849662])
  else:
      raise ValueError("Solo se permiten valores de grado entre 1 y 7")
  return xq, wq


## Cuadratura Gaussiana


  La función `gaussian_quad` implementa la fórmula de cuadratura Gauss-Legendre

  $$\int_{-1}^{1}f(x)dx \approx \sum_{i =1}^{q}w_{i}f(x_{i})$$
  
  Ésto evalúa la integral de `f` en el intervalo $[-1, 1]$ mediante los nodos (`xq`) y pesos (`wq`) previamente calculados.

**Entrada:**
    
*    `f : callable`

        Función a integrar. Debe ser evaluable en los puntos definidos
        por `xq`.
*    `xq : numpy.ndarray`

        Arreglo que contiene los nodos (raíces) del polinomio de Legendre
        en el intervalo [-1, 1].
*    `wq : numpy.ndarray`

        Arreglo que contiene los pesos asociados a los nodos, calculados
        según el grado de la cuadratura.

**Salida:**
    
* `result : float`
  Aproximación del valor de la integral definida de `f` en el intervalo $[-1, 1]$

    

In [62]:
def gaussian_quad(f, xq, wq):
  result = np.sum(f(xq) * wq)
  return result

## Transformación y su inversa


### Transformación



Transforma un intervalo estándar $[-1, 1]$ al intervalo $[a, b]$.
Esta transformación lineal es útil para aplicar cuadratura de Gauss en un intervalo arbitrario $[a, b]$, trasladando los nodos de la cuadratura estándar a este nuevo intervalo.

$$T(x) :=  \frac{(b-a)x+(b+a)}{2}$$

**Entrada:**
    
* `x : float o numpy.ndarray`

  Punto(s) en el intervalo estándar [-1, 1] que se desea transformar.

* ` a : float`

  Límite inferior del nuevo intervalo.
* ` b : float`

  Límite superior del nuevo intervalo.

**Salida:**

* `float o numpy.ndarray`

  Punto(s) transformados al intervalo $[a, b]$.

    


In [63]:
def T(x,a,b):
  return 0.5*(b-a)*x + 0.5*(b+a)

### Inversa


Transforma un punto del intervalo $[a, b]$ al intervalo estándar $[-1, 1]$. Es la inversa de la transformación `T`. Esto permite trabajar nuevamente en el dominio estándar después de realizar una transformación a $[a, b]$.

$$T^{-1}(x) = \frac{2}{b-a}\cdot\left(x-\frac{a+b}{2}\right)$$

**Entrada:**

*    `x : float o numpy.ndarray`

        Punto(s) en el intervalo [a, b] que se desea transformar de vuelta a [-1, 1].
*    `a : float`

        Límite inferior del intervalo original.

*    `b : float`

        Límite superior del intervalo original.

**Salida:**

*    `float o numpy.ndarray`

  Punto(s) transformados al intervalo estándar $[-1, 1]$


In [64]:
def T_inv(x,a,b):
  return (2/b-a)((x-a)-1)

## Definición de los Polinomios de Lagrange

Los polinomios de Lagrange son una forma de interpolación que usa una serie de puntos $(x_i,y_i)$. La fórmula general para el polinomio $L_{i}(x)$ está dada por

$$L_{i}(x):= \prod_{j=0 \\ j\neq i}^{n} \frac{x-x_{j}}{x_{i}-x_{j}}$$

En este caso, programar los polinomios implica un alto coste computacional, por ende tenemos que definir los polinomios de Lagrange y sus derivadas para $1\leq P\leq 3$ en base a consultas, pues con ello se reduce el coste computacional



### Funciones

In [65]:
#Polinomio grado 1
f01= lambda x:0.5*(1-x)
f11= lambda x:0.5*(1+x)

#Polinomio grado 2
f02= lambda x:0.5*x*(x-1)
f12= lambda x:-x*x + 1
f22= lambda x:0.5*x*(x+1)

#Polinomio grado 3
f03= lambda x:(-9/16)*x**3 + (9/16)*x**2 + (1/16)*x - 1/16
f13= lambda x:(27/16)*x**3 -  (9/16)*x**2 - (27/16)*x + 9/16
f23= lambda x:(-27/16)*x**3 - (9/16)*x**2 + (27/16)*x + 9/16
f33= lambda x:(9/16)*x**3 + (9/16)*x**2 - (1/16)*x - 1/16

### Derivadas

In [66]:
#Derivada grado 1
df01 = lambda x: -0.5
df11 = lambda x: 0.5

#Derivada grado 2
df02 = lambda x: 0.5 * (2 * x - 1)
df12 = lambda x: -2 * x
df22 = lambda x: x + 0.5

#Derivada grado 3
df03 = lambda x: -(27/16)*x**2 + 18/16*x + 1/16
df13 = lambda x: (81/16)*x**2 - (18/16)*x - 27/16
df23 = lambda x: -(81/16)*x**2 -(18/16)*x + 27/16
df33 = lambda x: (27/16)*x**2 + (18/16)*x - 1/16

### Creación de Diccionario

A partir de los datos sueltos, vamos a juntarlos en dos diccionarios, uno de para los polinomios que es `MD`, y otro para las derivadas (`dMD`)

In [67]:
#Diccionario de polinomios
MD = [
    {0: f01, 1: f11},
    {0: f02, 1: f12, 2: f22},
    {0: f03, 1: f13, 2: f23, 3: f33}
]

#Diccionario de derivadas
dMD = [
    {0: df01, 1: df11},
    {0: df02, 1: df12, 2: df22},
    {0: df03, 1: df13, 2: df23, 3: df33}
]

### Función definida de Lagrange y su derivada

Mediante esto, la fnción prácticamente será consultar los datos del diccionario.

#### Polinomio

 Evalúa el polinomio de Lagrange asociado al índice l y al grado p en el punto x.

**Entrada:**
*  `x : float`

      El punto donde se evaluará el polinomio.

*  `l : int`

  El índice del polinomio de Lagrange $0 \leq l \leq p$.

*  `p : int`

      El grado del polinomio $1\leq p \leq3$.

**Salida:**
    
* `f(x): float`

  Valor del polinomio de Lagrange en x.

In [68]:
def L(x,l,p):
  f=MD[p-1][l]
  return f(x)

#### Derivada

Evalúa la derivada del polinomio de Lagrange asociado al índice l y al grado p en el punto x.

**Entrada:**
*  `x : float`

  El punto donde se evaluará la derivada.
*  `l : int`

     El índice del polinomio de Lagrange $0  l \leq p$
*  `p : int`

   El grado del polinomio $1 \leq p \leq 3$

**Salida:**

*  `f(x):  float`


  Valor de la derivada del polinomio de Lagrange en x.


In [69]:
def dL(x,l,p):
  f=dMD[p-1][l]
  return f(x)

## Matrices de Masa y Rigidez

### Locales

#### De Masa


La matriz de masa local representa integrales de productos entre los polinomios de Lagrange $L_{i}(x)$ y $L_{j}(x)$, evaluados en un elemento. Por lo que el armado está dado por

$$M_{ij} = \int_{-1}^{1}L_{i}(x)\cdot L_{j}(x)$$

La simetría de la matriz se asegura computando una sola vez y asignando ambos valores cuando $M_{ij}\leftarrow M_{ji}$


**Entrada:**

* `p : int`

Grado del polinomio

* `xq, wq: numpy.ndarray`

Puntos y pesos de cuadratura de Gauss para la integración

**Salida:**
* `M : numpy.matrix`

Matriz de tamaño $(p+1)\times (p+1)$



In [70]:
def M_loc(p, xq, wq):
    #Inicializa una matriz local M para el almacenamiento de la matriz de masa
    M = np.zeros((p + 1, p + 1))
    #Rellena la matriz de masa integrando los productos de las funciones de base de Lagrange
    for i in range(p + 1):
      for j in range(p + 1):
        integrand = lambda x: L(x, i, p) * L(x, j, p)
        M[i, j] = gaussian_quad(integrand, xq, wq)
        M[j, i] = M[i, j]
    return M

#### De Rigidez

Por su parte, la matriz de rigidez local contiene las integrales del producto pero ahora de las derivadas de los polinomios de Lagrange, es totalmente análoga a la de rigidez

$$S_{ij} = \int_{-1}^{1}dL_{i}(x)\cdot dL_{j}(x)$$


**Entrada:**

* `p : int`

Grado del polinomio

* `xq, wq: numpy.ndarray`

Puntos y pesos de cuadratura de Gauss para la integración

**Salida:**

* `S : numpy.matrix`

Matriz de tamaño $(p+1)\times (p+1)$

In [71]:
def S_loc(p, xq, wq):
    # Inicializa una matriz local S para el almacenamiento de la matriz de rigidez
    S = np.zeros((p + 1, p + 1))
    # Rellena la matriz de rigidez integrando los productos de las derivadas de las funciones de base de Lagrange
    for i in range(p + 1):
        for j in range(p + 1):
            integrand = lambda x: dL(x, i, p) * dL(x, j, p)
            S[i, j] = gaussian_quad(integrand, xq, wq)
            S[j, i] = S[i, j]
    return S

### Globales $M,S$


**Entrada:**

- `p: int`  
  Grado del polinomio de base en cada elemento.  

- `x: list[float]`  
  Lista de coordenadas de los nodos globales que dividen el dominio.  

- `xq: list[float]`  
  Lista de puntos de cuadratura de Gauss-Legendre normalizados al intervalo $[-1, 1]$.  

- `wq: list[float]`
  Lista de pesos asociados a los puntos de cuadratura.  



**Salida:**


- `M: np.ndarray`

  Matriz de masa global de dimensiones $(Ne+1) \times (Ne+1)$  

- `S: np.ndarray`

  Matriz de rigidez global de dimensiones $(Ne+1) \times (Ne+1)$.  

In [72]:
def MS(p, x, xq, wq):
    N = len(x) - 1  # Número de elementos
    Ne = N * p  # Número de nodos globales es N*p + 1
    M, S = np.zeros((Ne+1, Ne+1)), np.zeros((Ne+1, Ne+1))  # Inicialización de matrices globales

    M_local = M_loc(p, xq, wq)
    S_local = S_loc(p, xq, wq)

    # Ciclo sobre los elementos para ensamblar
    for i in range(N):
      x_iL = x[i]               # Límite Izq
      x_iR = x[i + 1]           # Límite Der
      Ji = 0.5 * (x_iR - x_iL)  # Jacobiano del intervalo
      S_l = (1 / Ji) * S_local
      M_l =  Ji * M_local

      # Ensamblado global
      start = i * p
      end = start + p + 1

      # Sumar las matrices locales a las globales en los nodos correspondientes
      M[start:end, start:end] +=  M_l
      S[start:end, start:end] +=  S_l

    # Aplicación de condiciones de frontera de Dirichlet homogéneas
    M, S = M[1:-1, 1:-1], S[1:-1, 1:-1]
    return M, S


## Vector b

La función `RHS` calcula el vector del lado derecho (RHS, por sus siglas en inglés) global, que aparece en la formulación débil del problema. Este vector representa la contribución de la función fuente $ f $ en todo el dominio discretizado, integrando localmente en cada elemento y ensamblando los resultados en el vector global.

El proceso se realiza mediante:

1. **Transformación local a global:** Se considera cada elemento del dominio definido por los nodos en `x`, transformando los puntos locales de integración al intervalo global correspondiente usando la función `T`.

2. **Cálculo local:** Para cada par de nodos del elemento, se evalúan las contribuciones usando la base de polinomios de Lagrange \( L \) y se integran numéricamente mediante cuadratura de Gauss.

3. **Ensamblaje global:** Las contribuciones locales de cada elemento se agregan al vector global `b`, tomando en cuenta los índices correspondientes.

El resultado es un vector `b` que incorpora las contribuciones de la función \( f \) en toda la malla, listo para ser utilizado en la resolución del sistema lineal asociado al problema.



**Entrada:**

- `f: function`

  Función que representa la fuente en el problema (puede depender de \(x\)).  

- `x: list[float]`

  Lista de coordenadas de los nodos globales que dividen el dominio.  

- `p: int`

  Grado del polinomio de base en cada elemento.  

- `xq: list[float]`

  Lista de puntos de cuadratura de Gauss-Legendre normalizados al intervalo $[-1, 1]$.  

- `wq: list[float]`

  Lista de pesos asociados a los puntos de cuadratura.  


**Salida:**

- `b: np.ndarray`  
  Vector del lado derecho (RHS) global de dimensión $(Ne+1)$.  

In [73]:
def RHS(f, x, p, xq, wq):
    N = len(x) - 1        # Número de elementos
    Ne = N * p            # Número total de nodos (sin contar nodos duplicados)
    b = np.zeros(Ne + 1)  # Vector resultado inicializado en ceros

    for i in range(N):
        start = i * p        # Índice de inicio en el vector b
        end = start + p + 1  # Índice de fin en el vector b (p+1 nodos por elemento)

        x_iL = x[i]               # Límite izquierdo del intervalo
        x_iR = x[i + 1]           # Límite derecho del intervalo
        Ji = 0.5 * (x_iR - x_iL)  # Jacobiano del intervalo

        b_local = np.zeros(p + 1)  # Vector local para acumulación de resultados

        # Cuadratura de Gauss para cada función de base en el intervalo actual
        for k in range(p + 1):
            # Definimos el integrando fk
            fk = lambda w: Ji * f(T(w, x_iL, x_iR)) * L(T(w, x_iL, x_iR), k, p)
            # Realizamos la cuadratura de Gauss
            b_local[k] = gaussian_quad(fk, xq, wq)

        # Acumulamos el resultado en el vector global b
        b[start:end] += b_local

    return b


# Eigenvalores


El objetivo del código es el cálculo de los eigenvalores del operador diferencial, buscan resolver el problema:

$$Sv = \lambda M v$$

Dónde

* $S$ es la matriz de rigidez
* $M$ es la matriz de masa
* $\lambda$ son los eigenvalores
* $v$ son los eigenvectores asociados.

Los valores propios calculados sirven para aproximar el espectro discreto del operador diferencial asociado al problema.

Por otro lado, este còdigo nos sirve para evaluar cómo los valores propios convergen a medida que $dx\rightarrow0$, es decir, que el mallado se refina y se incrementa el grado del polinomio

### Condiciones Iniciales

In [75]:
xL=0
xR=np.pi
q=7
xq, wq = p_roots(q)
grados=[1,2,3]
resultados = []

In [76]:
for p in grados:
  for k in range(5):
    dx= np.pi*(0.5)**(k+3)
    N= int((xR-xL)/dx)
    x = np.linspace(xL,xR,N+1)
    y = np.linspace(xL,xR,(N*p)+1)
    M,S = MS(p, y, xq, wq)
    #Se calcula los valores propios resolviendo el problema generalizado S v = λ M v

    eigenvalores, _ = la.eigh(S, M)
    resultados.append({
            "p": p,
            "k": k,
            "dx": dx,
            "λ1": eigenvalores[0],
            "λ2": eigenvalores[1],
            "λ3": eigenvalores[2],
            "λ4": eigenvalores[3],
            "λ5": eigenvalores[4],
        })


# Se crea un DataFrame con los resultados
df_resultados = pd.DataFrame(resultados)
print(df_resultados)

    p  k        dx        λ1        λ2         λ3         λ4         λ5
0   1  0  0.392699  1.012916  4.209547  10.080291  19.453667  33.262830
1   1  1  0.196350  1.003217  4.051664   9.263131  16.838190  27.064923
2   1  2  0.098175  1.000803  4.012867   9.065245  16.206657  25.505923
3   1  3  0.049087  1.000201  4.003214   9.016276  16.051470  25.125749
4   1  4  0.024544  1.000050  4.000803   9.004067  16.012855  25.031390
5   2  0  0.392699  1.000002  4.000131   9.001478  16.008194  25.030734
6   2  1  0.196350  1.000000  4.000008   9.000094  16.000524  25.001991
7   2  2  0.098175  1.000000  4.000001   9.000006  16.000033  25.000126
8   2  3  0.049087  1.000000  4.000000   9.000000  16.000002  25.000008
9   2  4  0.024544  1.000000  4.000000   9.000000  16.000000  25.000000
10  3  0  0.392699  1.000000  4.000000   9.000000  16.000003  25.000019
11  3  1  0.196350  1.000000  4.000000   9.000000  16.000000  25.000000
12  3  2  0.098175  1.000000  4.000000   9.000000  16.000000  25